In [1]:
from __future__ import division
import itertools
import datetime
import hashlib
import scipy.spatial.distance as ssd
from sklearn.preprocessing import normalize
import locale
import numpy as np
import pycountry
import scipy.io as sio
import scipy.sparse as ss
from collections import defaultdict

#STEP 1-ceating user_event_response matrix 
#We are first creating emply sets to hold all the unique users and events
distinct_users = set()
distinct_events = set()
#Next we create empty dictionaries, one to hold all events presented to each user and another to hold all users each event is presented to.
events_for_each_user = defaultdict(set)
users_for_each_event = defaultdict(set)
#The below lines of code fill the two sets and two dictionaries with respective contents as mentioned above
for data in ["train.csv", "test.csv"]:
  file = open(data, 'r')
  file.readline().strip().split(",")
  #print(f)
  for line in file:
    columns = line.strip().split(",")
    distinct_users.add(columns[0])
    distinct_events.add(columns[1])
    events_for_each_user[columns[0]].add(columns[1])
    users_for_each_event[columns[1]].add(columns[0])
  file.close()
#dok_matrix is used to generate a sparse matrix(to keep only those entries that are non zero as row index,column index,value)
user_event_response = ss.dok_matrix((len(distinct_users), len(distinct_events)))
#Creating two dictionaries to store users and events with their indices.
user_index=dict()
event_index = dict()
#filling user_index with user:index
for i, l in enumerate(distinct_users):
  user_index[l] = i
#dilling event_index with event:index
for i, l in enumerate(distinct_events):
  event_index[l] = i

train = open("train.csv", 'r')
#skip the first row
train.readline()
#Here we are filling the sparse matrix: the sparse matrix contains: for each event-user in the training data-> the index of the user 
#and the index of the event according to the user_index and event_index list followed by user's response to the event
for l in train:
  col = l.strip().split(",")
  #print(col)
  i = user_index[col[0]]
  j = event_index[col[1]]
  user_event_response[i, j] = int(col[4]) - int(col[5])
#user's response to event will be 1 if he is interested , -1 is not interested , and 0 if neither.(but these 0's wont be written to the sparse matrix)
train.close()
#Now we are writing this sparse matrix to a mtx file
sio.mmwrite("user_event_response", user_event_response)


In [2]:

#STEP 2-finding unique user and event pairs
 
distinct_user_pairs = set()
distinct_event_pairs = set()
# Now we will find all unique user pairs and event pairs
#  These should be users who are linked via an event or events that are linked via a user in either the training or test sets. 
for e in distinct_events:
  u_list = users_for_each_event[e]
  if len(u_list) > 2:
    distinct_user_pairs.update(itertools.combinations(u_list, 2))

for u in distinct_users:
      e_list = events_for_each_user[u]
      if len(e_list) > 2:
        distinct_event_pairs.update(itertools.combinations(e_list, 2))
#________________________________________________________________________________________________________________________________________


# STEP-3 : Encoding the variables

# Locale is a column in users.csv. But it is categorical. We need to encode this. But we know that with new entries, there could 
#be new locales, so in order to encode it properly accounting all possible locales, we create a dictionary that contains all possible
#locales mapped to unique id's. We use the locale library for this
locales_id = defaultdict(int)
for i, loc in enumerate(locale.locale_alias.keys()):
  locales_id[loc] = i + 1
#print(localeIdMap)

# Similarly , we need to encode the location,again with new entries, there could 
#be new countries, so in order to encode it properly accounting all possible countries, we create a dictionary that contains all possible
#countries mapped to unique id's. We use the pycountry library for this
# load countries
country_id = defaultdict(int)
for i, country in enumerate(pycountry.countries):
  #print(c)
  country_id[country.name.lower()] = i + 1 
#print(countryIdMap)
#print(ctryIdx)

#To encode gender: male->1 female-> 2
gender_id = defaultdict(int, {"male":1, "female":2})
#print(genderIdMap)


#Following are the functions to encode each of the six variables in users.csv and events.csv
#locale is encoded using the locales_id dictionary created above
def encode_locale(l):
    return locales_id[l.lower()]
#gender is encoded using the gender_id dictionary created above
def encode_gender(g):
    return gender_id[g]

def encode_birth(b):
    try:
      return 0 if b == "None" else int(b)
    except:
      return 0

def encode_joined_month(d):
    val = datetime.datetime.strptime(d, "%Y-%m-%dT%H:%M:%S.%fZ")
    return "".join([str(val.year), str(val.month)])

def encode_timezone(t):
    try:
      return int(t)
    except:
      return 0

#location is encoded using the country_id dictionary created above
def encode_country(c):
    if (isinstance(c, str) and len(c.strip()) > 0 and c.rfind("  ") > -1):
      #print(location[location.rindex("  "):].lower())
      return country_id[c[c.rindex("  ") + 2:].lower()]
    else:
      return 0

def feature_hash(value):
    if len(value.strip()) == 0:
      return -1
    else:
      return int(hashlib.sha224(value).hexdigest()[0:4], 16)

def float_value(value):
    if len(value.strip()) == 0:
      return 0.0
    else:
      return float(value)
#____________________________________________________________________________


#STEP 4 : creating user matrix with index of each user and their preprocessed variables
n = len(user_index.keys())
#print(n)
file = open("users.csv", 'r')
columns = file.readline().strip().split(",")
#print(len(columns))
user_mat = ss.dok_matrix((n, len(columns) - 1)) 
for l in file:
  col = l.strip().split(",")
  # consider the users if they are in train.csv
  if col[0] in user_index:
    i = user_index[col[0]]
    user_mat[i, 0] = encode_locale(col[1])
    #print(encode_locale(col[1]))
    user_mat[i, 1] = encode_birth(col[2])
    user_mat[i, 2] = encode_gender(col[3])
    user_mat[i, 3] = encode_joined_month(col[4])
    user_mat[i, 4] = encode_country(col[5])
    user_mat[i, 5] = encode_timezone(col[6])
file.close()

sio.mmwrite("temp", user_mat)



In [4]:
#STEP 5: finding similarity between each pair of users
#Normalise the user matrix
user_mat = normalize(user_mat, norm="l1", axis=0, copy=False)
sio.mmwrite("user_matrix", user_mat)
# calculate the user similarity matrix
similarity_mat = ss.dok_matrix((n, n))
#diagonal elements are one as the similarity between a user and himself is 1
for i in range(0, n):
  similarity_mat[i, i] = 1.0
# for each unique pair in distinct_user_pairs, the correlation similarity measure is found by comparing the feature vectors for the users
#and that entry is stored in the similarity matrix
#this matrix is then written into the similaritymatrix.mtx file
for a,b in distinct_user_pairs:
  i = user_index[a]
  j = user_index[b]
  if similarity_mat[i,j]==0.0:
    coeff = ssd.correlation(user_mat.getrow(i).todense(),
      user_mat.getrow(j).todense())
    similarity_mat[i, j] = coeff
    similarity_mat[j, i] = coeff
sio.mmwrite("similaritymatrix", similarity_mat)

# To generate the event - event similarity matrix
# There are two main aspects that can be considered the event meta data and the event content based on which the similarity 
# matrix can be generated. So, we decide to generate two similarity matrices for the events one based on the event meta data
# for which the similarity measure used is correlation and the other based on the content which uses the cosine similarity measure.

# Reading the file for the processing.
fin = open("events.csv", 'r')
# Skipping the first line as it includes the column names.
fin.readline()
# Determining the number of distinct events.
n_evts = len(event_index.keys())
# Generating the sparse matrices to store the feature vectors for each event
# 1. Based on the event meta data
evt_prop_matrix = ss.dok_matrix((n_evts, 7))
# 2. Based on the event content
evt_cont_matrix = ss.dok_matrix((n_evts, 100))
z = 0
# Populating the previously created sparse matrices with the feature vectors corresponding to the distinct events by reading the events.csv file.
for line in fin.readlines():
    z += 1
    cols = line.strip().split(",")
    e_id = cols[0]
    if e_id in event_index:
        i = event_index[e_id]
        # Cleaning the data read from the file by using the functions defined previously
        # 1. Populating the event Properties matrix
        evt_prop_matrix[i, 0] = encode_joined_month(cols[2]) # start_time
        evt_prop_matrix[i, 1] = feature_hash(bytes(cols[3], "utf-8")) # city
        evt_prop_matrix[i, 2] = feature_hash(bytes(cols[4], "utf-8")) # state
        evt_prop_matrix[i, 3] = feature_hash(bytes(cols[5], "utf-8")) # zip
        evt_prop_matrix[i, 4] = feature_hash(bytes(cols[6], "utf-8")) # country
        evt_prop_matrix[i, 5] = float_value(cols[7]) # lat
        evt_prop_matrix[i, 6] = float_value(cols[8]) # lon
        # 2. Populating the event content matrix
        for j in range(9, 109):
            evt_cont_matrix[i, j-9] = cols[j]
fin.close()
# Normalizing the columns of the matrices generated to make sure that the values measured on the different scales 
# do no add unnecessary meaning or correlation.
# For both the matrices l1 norm or Manhattan distance is being used.
evt_prop_matrix = normalize(evt_prop_matrix, norm="l1", axis=0, copy=False)
sio.mmwrite("EV_eventPropMatrix", evt_prop_matrix)
evt_cont_matrix = normalize(evt_cont_matrix, norm="l1", axis=0, copy=False)
sio.mmwrite("EV_eventContMatrix", evt_cont_matrix)


In [5]:

# Objects that calculate the correlation and cosine similarity measure when two feature vectors are given
psim=ssd.correlation
csim=ssd.cosine

# Generating the similarity matrices to later populate with the similarity measures.
evt_prop_sim = ss.dok_matrix((n_evts, n_evts))
evt_cont_sim = ss.dok_matrix((n_evts, n_evts))

# Populating the similarity matrices by calculating the correlation and cosine similarity for the meta data based and content based
# matrices.
for e1, e2 in distinct_event_pairs:
    i = event_index[e1]
    j = event_index[e2]
    if evt_prop_sim[i,j] == 0.0:
        epsim = psim(evt_prop_matrix.getrow(i).todense(), evt_prop_matrix.getrow(j).todense())
        evt_prop_sim[i, j] = epsim
        evt_prop_sim[j, i] = epsim
    if evt_cont_sim[i,j] == 0.0:
        ecsim = csim(evt_cont_matrix.getrow(i).todense(), evt_cont_matrix.getrow(j).todense())
        evt_cont_sim[i, j] = ecsim
        evt_cont_sim[j, i] = ecsim
sio.mmwrite("EV_eventPropSim", evt_prop_sim)
sio.mmwrite("EV_eventContSim", evt_cont_sim)


#The friends of the specified user are found out 
#The idea is that: (a) people with more friends are more likely to attend events 
#(b) if your friend is going, its more likely for you to go as well


C:\Users\HP\Anaconda3\lib\site-packages\scipy\spatial\distance.py:505: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [11]:

#Determining the number of distinct users
nusers = len(user_index.keys())
numFriends = np.zeros((nusers))
userFriends = ss.dok_matrix((nusers, nusers))
#reading the data for preprocessing
#the user_friends.csv wasnt uploaded due to huge size
fin = open("user_friends.csv", 'r')
# skip header
fin.readline()  
ln = 0
for line in fin:
      cols = line.strip().split(",")
      user = cols[0]
      print(user)
      if user in user_index :
        friends = cols[1].split(" ")
        i = user_index[user]
        numFriends[i] = len(friends)
        for friend in friends:
          if friend in user_index :
            j = user_index[friend]
            print(friend)
            # the objective of this score is to infer the degree to
            # and direction in which this friend will influence the
            # user's decision, so we sum the user/event score for
            # this user across all training events.
            eventsForUser =user_event_response.getrow(j).todense()
            score = eventsForUser.sum() / np.shape(eventsForUser)[1]
            userFriends[i, j] += score
            userFriends[j, i] += score
        ln += 1
print("done")
fin.close()
# normalize the arrays
sumNumFriends =numFriends.sum(axis=0)
numFriends = numFriends / sumNumFriends
sio.mmwrite("num_friends", np.matrix(numFriends))
#using the l1 norm or the manhattan distance
userFriends = normalize(userFriends, norm="l1", axis=0, copy=False)
sio.mmwrite("user_friends", userFriends)



3197468391
3537982273
823183725
1872223848
3429017717
627175141
2752000443
3473687777
2966052962
264876277
1534483818
2648135297
1355996271
2411726276
1692345316
3786316231
2081587
184647001
1013376584
2686249984
2276008938
1091720544
3593684261
839960040
3184414062
84133998
851286067
3341263967
1363661070
1911472832
1186129084
3704022334
2396669325
2370879614
1748125486
3481126833
687155152
3104149470
4003794851
1000773879
4091235322
1889728955
3182865964
486770608
3217740197
1767910530
3981164786
2909448722
3498595321
1518185911
2024389351
4272286295
639980269
3276242622
1583891911
14952708
2038562858
2401908633
1220889821
983316945
772134053
1158367242
3359566214
2793136619
3241260218
3171083451
4101519751
2568724000
3076110891
1638865939
1326153915
1622358303
556235032
2084209520
3241496548
2163304477
2536442907
3213877831
516112393
1185293178
2521576959
3301922958
2760353297
478822209
400614728
1257044723
1664144722
3014754205
3907797051
231604513
397493741
693819894
1066251866
69

2869467989
3009215126
2419426713
2272339133
4017291216
1646256245
2171680297
1475858600
2674117778
251071109
2384368721
478223774
1704437471
2137883118
3548703208
1244880349
3296890774
3419383268
2388589122
1260511469
3687967272
361223314
1653042659
3123500032
1224749340
3338817729
1325052851
2902208687
1296007357
3711334312
3375265240
1567970024
610133863
2937936158
711114701
1276180920
4215261103
101139532
3420765349
2119839197
4110643370
4157885496
1669438538
170812873
2189497721
3316923302
4181543522
76794098
1456300969
3437707705
1355394873
1233029527
1284435177
896794819
946618459
2637352610
3435925476
2564389489
4098190588
1340520017
3924427919
2287869902
1638271360
382829431
2033240422
3871561495
2345632262
77348873
3553494760
768519060
174097564
3547678767
95505162
3261424687
1079339663
4244109605
3834655845
3035853030
414832767
1925090585
4163629094
3029719272
1490282397
2107966400
959869165
863165630
3561509722
4004924535
2144746556
677882200
4042925768
401171905
772508495
4

3627636732
2330978075
3483447342
243341329
779020543
640550522
2054648409
2903766764
212294629
2916559717
3599572670
3350077468
683072052
3577102921
1566352766
2040962701
2824895709
3875142269
4131316066
3934518980
2611239561
1784363573
1178274219
3176283501
2944601898
2007755085
3143612077
1834315541
446963892
4070364811
2393251888
70585535
2176903883
718524754
1393629953
3290568694
812562995
3851487388
1488540499
1338104877
1812718144
3349277777
4176372154
576889228
180720092
4194275561
3775565693
2918748163
2114948987
3198801990
2469046391
2388007096
3191212256
1684440449
2912402757
927009190
3323166723
3771702031
1649320589
293347866
1404876129
1919112590
1917763717
3305922780
1040573189
1859654041
828240618
1683199039
2959168426
1619810899
758847379
2295470173
1162995930
664267673
4164394451
1336539109
1033472318
1889167056
3821168737
155107844
2318294517
967361915
2692979972
282914818
2744115938
3823320698
2986164966
1216094400
3760170654
1892392734
3282478477
1910721259
17894185

2012972911
248375618
1077624325
2944574958
341519370
2383553413
2847066139
844585350
1847788691
2638473114
870261804
3866940390
1531532175
2471225429
3856393866
1020462518
2396135511
1533222368
36963490
3304089510
1854913858
4121491618
1473654738
2724319931
1049146719
620761021
1039126403
4088211339
3724535659
3838242688
2633145288
4277254546
1633852237
329870864
1030938011
1301869393
2910476131
4089415788
3945141646
3480781792
249918369
2301837429
1821614210
2811699811
2770315632
4015274605
3346325459
1635038587
3778269624
1526909321
481693470
3357420534
1535249805
543735376
2317932056
2685979734
2010818203
742551318
1912009669
3909997753
43026617
1699374488
4104050557
408136590
2475037548
2667210877
3837379530
2675088061
3855705749
2871958842
3630567426
4115435379
1834081867
802215855
87423860
864140472
858124113
3288090548
4079321652
2251621643
3486848738
60259409
3754081612
2029494385
1506493101
3855083472
2813614906
3261989043
3319127416
916619191
4257258690
688722024
998505128
16

3962767729
2706413631
2884249863
3647725503
2505058685
4151447403
4082018247
2513403790
1854575078
2300798207
1361841503
1811195921
2126004440
2157346824
3288440219
2694632667
566640799
3229359863
1451393971
3783631177
2793916391
3412293331
3561286101
3686254404
2812778246
4147581285
3204451544
2753765193
3295164681
882260183
2928561788
3715799927
3804758314
2408913046
3990201798
3358895762
1414265901
1017119096
3480840234
2100831942
630543733
1323318123
4294027964
813877428
3568907921
1484078818
188875346
4109397627
163558597
3534407842
887318586
1873452348
1036021683
1142873063
2333524603
1244391408
672063363
1151039938
220983599
2280079154
1747471686
1126868438
867285031
209839596
2400978518
776235729
1573596269
2208651876
2441832194
439279583
4115365036
1258740073
1759431765
3669515588
396008921
384932005
2697325655
791872426
2913615602
2200050655
1509615404
931663795
860563181
195488034
2995978907
2535162187
2049228429
3049806608
830807519
138231515
3468617687
424827596
1081617389

...

2755154178
2927772127
460024134
776025327
966430197
1028046987
2011291053
2785871963
2198198554
3490563315
697773199
3509192472
2617944784
747065527
3414473118
2066747365
264855431
1194037989
73009312
596044827
1196973493
1919183532
489379329
2952289181
193920233
4060088745
2394466749
471637420
2932100756
3186577794
2679652649
3868815488
2921976592
751215617
2872854546
2600681841
2236631339
587750031
3806010541
4076221689
171071695
4084567101
1287830174
3057038899
2646420550
1856111469
217940328
2898698909
2958038938
3055336890
1856054737
1020875748
4239920443
1600931671
4167954255
3641159022
3052778890
1169270868
1491636315
2647061724
2169857474
332948179
1774492410
4246181462
2754972313
1409896307
2391290359
3882650869
1241121575
3257385051
2891592919
2440481389
3114166378
3746792233
3493298001
780238213
107783241
1568609352
1783321549
776025327
3148555709
245442083
3481754148
2212919969
2927772127
2046441478
797637405
2166450955
1465564492
69835441
44612096
1281839953
2047409960
320

3594879390
1285386563
3952924915
713655475
2785699576
260442129
3836404628
3531831441
1377670966
1196840282
1543396188
2993674473
3376988073
3958701071
3494187068
4070988148
854400578
4065772987
1769475808
1965944390
1320099877
122743553
3513865352
3527084294
3471432441
2034906109
1530029005
4033835330
1762206332
2944688994
1175301793
142897219
1008893291
2316950688
351713549
617013723
1256077486
1572769592
1133408994
437509880
430743685
3826517645
360888868
1091610794
1637866930
699420462
3535468775
1578039954
2919350882
448144551
811243178
3310233227
2809968215
1717744516
975334672
905051781
3918040635
3887088577
3716046610
2675683493
1008895268
449269668
2934707117
2459103666
1386381159
3039544284
3190229663
122113785
3718242378
3996979962
4058274528
2739423793
1478667747
3099975181
327215752
2329955796
2890594749
3845256408
2973757966
1388035869
896063991
3856860345
3470008003
1584810552
1761488321
174843693
842417539
1717011781
1872977766
1645535779
3826799385
2318198315
233848153

1562897104
2274473064
1723637189
1857390350
428619631
2755154178
2892947506
2345460195
486162423
3497051269
1505890709
1627419158
3752474458
1253964401
2693584655
3691025600
1158456078
1853614834
423808640
638007435
3763649571
3558311890
507546100
3853283705
2367164749
2221494569
2851886820
257828961
2273559629
2220062960
2154153423
2999798867
2154482496
1800369258
3884291074
2961175089
1512365138
3681938868
2098969341
1564177449
781399360
3995782028
2045073099
3498417316
2042244835
526846242
2225266004
3343873693
993240761
2606857819
4029968122
608443709
2867658950
3898600272
1971105644
4167526213
154318109
910846055
1610565112
267940780
3749690658
974068849
3031311007
1724694030
4232243965
1771771365
1355363121
2249179961
3947744988
873346136
1271596933
382146836
4233334268
3903571397
3946799015
4070256540
3360001388
1166782297
772766380
552349486
159212671
3928458441
3951470738
428106660
1637956476
2124505909
3426525881
3645125918
1808101898
2251278782
2037662222
429612304
261485179

3099611687
208659737
3787502191
984507898
3494721226
3924307888
3078386697
1252230556
3010963141
1072103214
1714657910
3519205792
3659421361
907349714
1339650581
847230028
2848164993
3911391880
491263047
3444618080
3445873460
1908507314
2316226252
4235140075
1678909329
2435916709
1623287180
70068607
65329528
3295884480
1843206997
2062668000
1843222701
2969333698
2491957982
2869450440
2243396544
844324358
1826503918
3240481322
2627456213
2817511682
1245694497
1554070120
689961313
2841225823
4175659797
3370560295
2553806189
507590029
1184737011
3221482892
1166204486
2495407859
1644052705
1824088561
475081249
3875068822
3603006634
253594379
1606717488
2667572564
1669841406
3803408133
177883032
3717838008
3655537215
4220379428
1243553819
342778753
2320854059
2945726385
3427391302
905852095
1990497157
424866781
3279257076
1571871667
4098096768
3669048335
569826712
113021847
511678471
2813858112
1549535085
1107424122
2536109970
602148753
1482478167
3475966810
3874274652
1930383423
3892691592

367845178
1985341446
4054425557
3030655335
3497012580
1042981269
856293699
4162525077
1939990182
337604618
1100637880
2994496125
2796351540
3005358451
2107704029
2119640687
841838476
3830192097
2776460685
3113935310
2158526950
21348393
4054702657
3392108379
2914084325
529086617
1064472095
901305079
496936219
2985559658
602148753
3903245015
2060596237
917166122
1038828491
508925891
395230304
960150210
3334976367
3267210147
757828289
782816363
3896173460
701370956
3771776258
2057570125
2480756569
2778114948
1748926147
3451447483
2290106807
804076312
3536923238
676791111
1833392034
2507247484
3212362283
1940954160
1600931671
1260240845
567534910
4239920443
1344492095
1206686094
4167954255
2102853216
39843985
2142341019
169371285
2268207841
1515521305
2531508215
113424397
1159884840
2342591889
3118907135
40850721
1581246889
1503668152
2783945125
1753157101
119602679
231769845
2335809789
1562114120
3848744687
2638307735
3530946278
2351341132
190366480
1298916460
4242612936
3712460494
311863

In [18]:
import scipy.sparse as ss
import scipy.io as sio
nevents = len(event_index)
event_popularity = ss.dok_matrix((nevents, 1))
f = open("event_attendees.csv", 'r')
f.readline()
for line in f:
    cols = line.strip().split(",")
    event_id = cols[0]
    if event_id in event_index:
        i = event_index[event_id]
        event_popularity[i, 0] = len(cols[1].split(" ")) + 0.5*len(cols[2].split(" ")) - len(cols[4].split(" "))
f.close()
event_popularity = normalize(event_popularity, norm="l1", axis=0, copy=False)
sio.mmwrite("EA_eventPopularity", event_popularity)


In [19]:

import _pickle as cPickle
cPickle.dump(user_index, open("user_index.pkl", 'wb'))
cPickle.dump(event_index, open("event_index.pkl", 'wb'))